In [2]:
import networkx as nx
from collections import defaultdict

class NCCTracker:
    def __init__(self):
        # Initialize the residual graph with a source/sink node
        self.residual_graph = nx.DiGraph()
        self.residual_graph.add_node('s')  # Source/sink node
        self.fragment_counter = 0
        
    def add_fragment(self, fragment):
        """Add a new fragment to the graph and update associations"""
        fragment_id = self.fragment_counter
        self.fragment_counter += 1
        
        print(f"\n=== Adding Fragment {fragment_id} ===")
        print(f"Fragment details: Lane {fragment['lane']}, Time {fragment['start_time']}-{fragment['end_time']}")
        
        # Create entry and exit nodes for this fragment
        u_node = f'u{fragment_id}'
        v_node = f'v{fragment_id}'
        
        # Add nodes to residual graph
        self.residual_graph.add_node(u_node, fragment=fragment)
        self.residual_graph.add_node(v_node, fragment=fragment)
        
        # Connect source to entry node
        self.residual_graph.add_edge('s', u_node, cost=2, flow=0, capacity=1)
        print(f"Added edge: s -> {u_node} (cost=2, flow=0/1)")
        
        # Connect entry to exit node (include the fragment)
        self.residual_graph.add_edge(u_node, v_node, cost=0, flow=0, capacity=1)
        print(f"Added edge: {u_node} -> {v_node} (cost=0, flow=0/1)")
        
        # Connect exit node to sink
        self.residual_graph.add_edge(v_node, 's', cost=2, flow=0, capacity=1)
        print(f"Added edge: {v_node} -> s (cost=2, flow=0/1)")
        
        # Connect this fragment to previous fragments
        self._add_connections(fragment_id, fragment)
        
        # Print the current state of the residual graph
        self._print_residual_graph()
        
        # Find and cancel negative cycles
        self._find_and_cancel_negative_cycles()
        
        return fragment_id
    
    def _add_connections(self, current_id, current_fragment):
        """Add transition edges from existing fragments to this new fragment"""
        print("\nAdding transition edges:")
        
        for node in list(self.residual_graph.nodes()):
            if node.startswith('v') and node != f'v{current_id}':
                # This is an exit node of another fragment
                prev_id = int(node[1:])
                prev_node_data = self.residual_graph.nodes[f'u{prev_id}']
                
                if 'fragment' in prev_node_data:
                    prev_fragment = prev_node_data['fragment']
                    
                    # Calculate transition cost based on the fragments
                    cost = self._calculate_transition_cost(prev_fragment, current_fragment)
                    
                    # Add transition edge
                    self.residual_graph.add_edge(node, f'u{current_id}', cost=cost, flow=0, capacity=1)
                    print(f"Added edge: {node} -> u{current_id} (cost={cost}, flow=0/1)")
    
    def _calculate_transition_cost(self, prev_fragment, current_fragment):
        """Calculate transition cost between fragments"""
        time_diff = current_fragment['start_time'] - prev_fragment['end_time']
        lane_diff = abs(current_fragment['lane'] - prev_fragment['lane'])
        
        # Small time gap and same lane: negative cost (encouraging association)
        if lane_diff == 0:
            return -5 + 0.1 * time_diff  # Negative cost to encourage association
        else:
            return 10 + 0.1 * time_diff  # High cost for lane change
    
    def _find_and_cancel_negative_cycles(self):
        """Find and cancel all negative cycles in the residual graph"""
        print("\nLooking for negative cycles...")
        
        cycle_count = 0
        while True:
            # Find a negative cycle
            cycle = self._find_negative_cycle()
            if not cycle:
                print("No more negative cycles found.")
                break
                
            cycle_count += 1
            print(f"\n--- Found Negative Cycle #{cycle_count} ---")
            
            # Push flow through the cycle
            self._push_flow_through_cycle(cycle)
            
            # Print the updated residual graph
            self._print_residual_graph()
    
    def _find_negative_cycle(self):
        """Find a negative cycle in the residual graph"""
        # Create a graph with only cost as edge weights
        cost_graph = nx.DiGraph()
        
        for u, v, data in self.residual_graph.edges(data=True):
            if data.get('flow', 0) < data.get('capacity', 0):
                # Forward edge with residual capacity
                cost_graph.add_edge(u, v, weight=data['cost'])
            if data.get('flow', 0) > 0:
                # Backward edge with residual capacity
                cost_graph.add_edge(v, u, weight=-data['cost'])
        
        # Try to find a negative cycle using a simple approach
        for node in cost_graph.nodes():
            try:
                cycle = nx.find_negative_cycle(cost_graph, node)
                if cycle:
                    # Ensure it's a proper cycle (start and end node are the same)
                    if len(cycle) > 2 and cycle[0] == cycle[-1]:
                        total_cost = sum(cost_graph[cycle[i]][cycle[i+1]]['weight'] for i in range(len(cycle)-1))
                        print(f"Cycle found: {' -> '.join(cycle)} (total cost: {total_cost})")
                        return cycle
            except:
                continue
        
        # Manual cycle finding for simple cases
        for cycle in nx.simple_cycles(cost_graph):
            if len(cycle) > 2:  # Ensure it's a proper cycle
                # Add the first node again to make a complete cycle
                cycle.append(cycle[0])
                total_cost = sum(cost_graph[cycle[i]][cycle[i+1]]['weight'] for i in range(len(cycle)-1))
                if total_cost < 0:
                    print(f"Cycle found: {' -> '.join(cycle)} (total cost: {total_cost})")
                    return cycle
        
        return None
    
    def _push_flow_through_cycle(self, cycle):
        """Push one unit of flow through the cycle"""
        # Find minimum residual capacity in the cycle
        min_residual = float('inf')
        
        # Convert cycle to a list of edges
        edges = [(cycle[i], cycle[i+1]) for i in range(len(cycle)-1)]
        
        print("\nPushing flow through cycle:")
        print("Examining edge capacities:")
        
        # Calculate residual capacity for each edge
        for u, v in edges:
            if self.residual_graph.has_edge(u, v):
                # Forward edge
                residual = self.residual_graph[u][v]['capacity'] - self.residual_graph[u][v]['flow']
                print(f"  Edge {u} -> {v}: residual capacity = {residual}")
                min_residual = min(min_residual, residual)
            else:
                # Backward edge (look for the reverse edge)
                if self.residual_graph.has_edge(v, u):
                    residual = self.residual_graph[v][u]['flow']
                    print(f"  Edge {u} -> {v} (reverse of {v} -> {u}): residual capacity = {residual}")
                    min_residual = min(min_residual, residual)
                else:
                    print(f"  Error: Edge {u} -> {v} not found in either direction")
        
        # Push flow through the cycle
        print(f"\nPushing flow of {min_residual} through the cycle:")
        for u, v in edges:
            if self.residual_graph.has_edge(u, v):
                # Forward edge
                self.residual_graph[u][v]['flow'] += min_residual
                print(f"  Increased flow on {u} -> {v} to {self.residual_graph[u][v]['flow']}/{self.residual_graph[u][v]['capacity']}")
            else:
                # Backward edge (decrease flow on the reverse edge)
                if self.residual_graph.has_edge(v, u):
                    self.residual_graph[v][u]['flow'] -= min_residual
                    print(f"  Decreased flow on {v} -> {u} to {self.residual_graph[v][u]['flow']}/{self.residual_graph[v][u]['capacity']}")
    
    def _print_residual_graph(self):
        """Print the current state of the residual graph"""
        print("\nCurrent Residual Graph:")
        print("Nodes:", self.residual_graph.nodes())
        print("Edges:")
        
        for u, v, data in sorted(self.residual_graph.edges(data=True)):
            print(f"  {u} -> {v}: cost={data['cost']}, flow={data['flow']}/{data['capacity']}")
    
    def get_trajectories(self):
        """Extract trajectories from the current flow"""
        print("\n=== Extracting Final Trajectories ===")
        
        # Each trajectory is a sequence of fragments
        trajectories = []
        
        # Start with all entry nodes
        entry_nodes = [node for node in self.residual_graph.nodes() if node.startswith('u')]
        visited = set()
        
        for start_node in entry_nodes:
            if start_node in visited:
                continue
            
            # Check if this fragment is included (flow from entry to exit)
            fragment_id = int(start_node[1:])
            exit_node = f'v{fragment_id}'
            
            if not self.residual_graph.has_edge(start_node, exit_node) or self.residual_graph[start_node][exit_node]['flow'] == 0:
                continue
                
            # Start a new trajectory
            trajectory = []
            current_node = start_node
            
            print(f"\nStarting new trajectory from {current_node}:")
            
            # Follow the flow from entry to exit to next entry, etc.
            while current_node not in visited:
                visited.add(current_node)
                
                if current_node.startswith('u'):
                    fragment_id = int(current_node[1:])
                    trajectory.append(fragment_id)
                    print(f"  Added fragment {fragment_id} to trajectory")
                    
                    # Move to the exit node
                    exit_node = f'v{fragment_id}'
                    current_node = exit_node
                else:
                    # We're at an exit node, find the next fragment
                    found_next = False
                    for _, next_node, data in self.residual_graph.out_edges(current_node, data=True):
                        if data['flow'] > 0 and next_node != 's' and next_node.startswith('u'):
                            print(f"  Following flow from {current_node} to {next_node}")
                            current_node = next_node
                            found_next = True
                            break
                    
                    if not found_next:
                        # End of trajectory
                        print(f"  End of trajectory (no more flow from {current_node})")
                        break
            
            if trajectory:
                trajectories.append(trajectory)
        
        return trajectories

# Example usage
def main():
    tracker = NCCTracker()
    
    # Create some example fragments
    fragments = [
        {'id': 0, 'start_time': 0, 'end_time': 10, 'lane': 1, 'position': [0, 100]},
        {'id': 1, 'start_time': 0, 'end_time': 10, 'lane': 2, 'position': [0, 200]},
        {'id': 2, 'start_time': 15, 'end_time': 25, 'lane': 1, 'position': [200, 300]},
        {'id': 3, 'start_time': 15, 'end_time': 25, 'lane': 2, 'position': [200, 400]}
    ]
    
    # Add fragments one by one
    for fragment in fragments:
        fragment_id = tracker.add_fragment(fragment)
        print(f"\nFragment {fragment_id} added and processed.\n")
    
    # Get the final trajectories
    trajectories = tracker.get_trajectories()
    print("\nFinal Trajectories:")
    for i, traj in enumerate(trajectories):
        print(f"Trajectory {i+1}: Fragments {traj}")

if __name__ == "__main__":
    main()


=== Adding Fragment 0 ===
Fragment details: Lane 1, Time 0-10
Added edge: s -> u0 (cost=2, flow=0/1)
Added edge: u0 -> v0 (cost=0, flow=0/1)
Added edge: v0 -> s (cost=2, flow=0/1)

Adding transition edges:

Current Residual Graph:
Nodes: ['s', 'u0', 'v0']
Edges:
  s -> u0: cost=2, flow=0/1
  u0 -> v0: cost=0, flow=0/1
  v0 -> s: cost=2, flow=0/1

Looking for negative cycles...
No more negative cycles found.

Fragment 0 added and processed.


=== Adding Fragment 1 ===
Fragment details: Lane 2, Time 0-10
Added edge: s -> u1 (cost=2, flow=0/1)
Added edge: u1 -> v1 (cost=0, flow=0/1)
Added edge: v1 -> s (cost=2, flow=0/1)

Adding transition edges:
Added edge: v0 -> u1 (cost=9.0, flow=0/1)

Current Residual Graph:
Nodes: ['s', 'u0', 'v0', 'u1', 'v1']
Edges:
  s -> u0: cost=2, flow=0/1
  s -> u1: cost=2, flow=0/1
  u0 -> v0: cost=0, flow=0/1
  u1 -> v1: cost=0, flow=0/1
  v0 -> s: cost=2, flow=0/1
  v0 -> u1: cost=9.0, flow=0/1
  v1 -> s: cost=2, flow=0/1

Looking for negative cycles...
No 

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

class NCCTracker:
    def __init__(self):
        # Initialize the residual graph with a source/sink node
        self.residual_graph = nx.DiGraph()
        self.residual_graph.add_node('s', pos=(0, 0))  # Source/sink node
        self.fragment_counter = 0
        self.figure_counter = 0
        
    def add_fragment(self, fragment):
        """
        Add a new fragment to the graph.
        A fragment should have properties like start_time, end_time, position, etc.
        """
        fragment_id = self.fragment_counter
        self.fragment_counter += 1
        
        print(f"\n=== Adding Fragment {fragment_id} ===")
        print(f"Fragment details: Lane {fragment['lane']}, Time {fragment['start_time']}-{fragment['end_time']}")
        
        # Create entry and exit nodes for this fragment
        u_node = f'u{fragment_id}'
        v_node = f'v{fragment_id}'
        
        # Add nodes to residual graph with positions for visualization
        y_pos = fragment_id
        self.residual_graph.add_node(u_node, pos=(1, y_pos), fragment=fragment)
        self.residual_graph.add_node(v_node, pos=(2, y_pos), fragment=fragment)
        
        # Connect source to entry node
        self.residual_graph.add_edge('s', u_node, cost=2, flow=0, capacity=1)
        
        # Connect entry to exit node (include the fragment)
        self.residual_graph.add_edge(u_node, v_node, cost=0, flow=0, capacity=1)
        
        # Connect exit node to sink
        self.residual_graph.add_edge(v_node, 's', cost=2, flow=0, capacity=1)
        
        # Connect this fragment to previous fragments
        self._add_connections(fragment_id, fragment)
        
        # Visualize G+r,k before finding negative cycles
        if fragment_id > 0:
            self._visualize_residual_graph(
                title=f"G⁺ᵣ,{fragment_id-1} (Before Adding Fragment {fragment_id})",
                show_min_cost_circulation=True
            )
        
        # Find negative cycles, showing G-r,k+1 with negative cycle if found
        negative_cycle = self._find_negative_cycle()
        if negative_cycle:
            self._visualize_residual_graph(
                title=f"G⁻ᵣ,{fragment_id} (After Adding Fragment {fragment_id})",
                highlight_negative_cycle=negative_cycle
            )
            self._push_flow_through_cycle(negative_cycle)
        
        # Visualize G+r,k+1 after finding and canceling negative cycles
        self._visualize_residual_graph(
            title=f"G⁺ᵣ,{fragment_id} (After Processing Fragment {fragment_id})",
            show_min_cost_circulation=True
        )
        
        return fragment_id
    
    def _add_connections(self, current_id, current_fragment):
        """Add transition edges from existing fragments to this new fragment"""
        for node in self.residual_graph.nodes():
            if node.startswith('v') and node != f'v{current_id}':
                # This is an exit node of another fragment
                prev_id = int(node[1:])
                prev_node_data = self.residual_graph.nodes[f'u{prev_id}']
                
                if 'fragment' in prev_node_data:
                    prev_fragment = prev_node_data['fragment']
                    
                    # Calculate transition cost based on the fragments
                    cost = self._calculate_transition_cost(prev_fragment, current_fragment)
                    
                    # Add transition edge
                    self.residual_graph.add_edge(node, f'u{current_id}', cost=cost, flow=0, capacity=1)
    
    def _calculate_transition_cost(self, prev_fragment, current_fragment):
        """
        Calculate transition cost between fragments.
        Lower cost means higher likelihood of association.
        """
        # This is a simplified cost model. In practice, you'd use a more sophisticated model
        # based on position, velocity, appearance, etc.
        
        time_diff = current_fragment['start_time'] - prev_fragment['end_time']
        lane_diff = abs(current_fragment['lane'] - prev_fragment['lane'])
        
        # Small time gap and same lane: negative cost (encouraging association)
        if lane_diff == 0:
            return -5 + 0.1 * time_diff  # Negative cost for same lane
        else:
            return 10 + 0.1 * time_diff  # High cost for lane change
    
    def _find_negative_cycle(self):
        """Find a negative cycle in the residual graph using Bellman-Ford algorithm"""
        # Create a graph with only cost as edge weights
        cost_graph = nx.DiGraph()
        
        for u, v, data in self.residual_graph.edges(data=True):
            if data.get('flow', 0) < data.get('capacity', 0):
                # Forward edge with residual capacity
                cost_graph.add_edge(u, v, weight=data['cost'])
            if data.get('flow', 0) > 0:
                # Backward edge with residual capacity
                cost_graph.add_edge(v, u, weight=-data['cost'])
        
        # Try to find negative cycles using simple cycle enumeration
        for cycle in nx.simple_cycles(cost_graph):
            if len(cycle) > 2:  # Ensure it's a proper cycle
                # Add first node to close the cycle
                closed_cycle = cycle + [cycle[0]]
                # Calculate total cost
                total_cost = sum(cost_graph[closed_cycle[i]][closed_cycle[i+1]]['weight'] 
                                for i in range(len(closed_cycle)-1))
                if total_cost < 0:
                    print(f"Found negative cycle: {' -> '.join(closed_cycle)} with cost {total_cost}")
                    return closed_cycle
        
        print("No negative cycles found.")
        return None
    
    def _push_flow_through_cycle(self, cycle):
        """Push one unit of flow through the cycle"""
        # Find minimum residual capacity in the cycle
        min_residual = float('inf')
        
        # Convert cycle to a list of edges
        edges = [(cycle[i], cycle[i+1]) for i in range(len(cycle)-1)]
        
        # Calculate residual capacity for each edge
        for u, v in edges:
            if self.residual_graph.has_edge(u, v):
                # Forward edge
                residual = self.residual_graph[u][v]['capacity'] - self.residual_graph[u][v]['flow']
                min_residual = min(min_residual, residual)
            else:
                # Backward edge
                if self.residual_graph.has_edge(v, u):
                    residual = self.residual_graph[v][u]['flow']
                    min_residual = min(min_residual, residual)
        
        print(f"Pushing flow of {min_residual} through cycle: {' -> '.join(cycle)}")
        
        # Push flow through the cycle
        for u, v in edges:
            if self.residual_graph.has_edge(u, v):
                # Forward edge
                self.residual_graph[u][v]['flow'] += min_residual
            else:
                # Backward edge
                if self.residual_graph.has_edge(v, u):
                    self.residual_graph[v][u]['flow'] -= min_residual
    
    def _visualize_residual_graph(self, title=None, highlight_negative_cycle=None, show_min_cost_circulation=False):
        """
        Visualize the current state of the residual graph with specified formatting.
        
        Parameters:
        - title: Title for the graph
        - highlight_negative_cycle: If provided, highlight this cycle in red dashed lines
        - show_min_cost_circulation: If True, highlight the min-cost circulation in bold black lines
        """
        self.figure_counter += 1
        plt.figure(figsize=(12, 8))
        
        # Get node positions
        pos = nx.get_node_attributes(self.residual_graph, 'pos')
        
        # If any node doesn't have a position, use spring layout
        if len(pos) < len(self.residual_graph.nodes()):
            missing_nodes = [n for n in self.residual_graph.nodes() if n not in pos]
            pos_subset = nx.spring_layout(self.residual_graph.subgraph(missing_nodes))
            pos.update(pos_subset)
        
        # Draw all nodes
        nx.draw_networkx_nodes(self.residual_graph, pos, node_size=500, 
                            node_color='lightblue', alpha=0.8)
        
        # Prepare for different edge styles
        circulation_edges = []
        potential_edges = []
        edge_labels = {}
        
        for u, v, data in self.residual_graph.edges(data=True):
            # Add edge label
            edge_labels[(u, v)] = f"{data['flow']}/{data['capacity']}\n({data['cost']})"
            
            # Categorize edges
            if data['flow'] > 0:
                circulation_edges.append((u, v))
            else:
                potential_edges.append((u, v))
        
        # Draw potential edges (dashed black)
        nx.draw_networkx_edges(self.residual_graph, pos, edgelist=potential_edges,
                           width=1.0, edge_color='black', style='dashed',
                           arrows=True, arrowsize=15)
        
        # Draw circulation edges (solid black)
        if circulation_edges:
            nx.draw_networkx_edges(self.residual_graph, pos, edgelist=circulation_edges,
                               width=2.0, edge_color='black',
                               arrows=True, arrowsize=15)
        
        # If requested, highlight the min-cost circulation (bold solid black)
        if show_min_cost_circulation and circulation_edges:
            nx.draw_networkx_edges(self.residual_graph, pos, edgelist=circulation_edges,
                               width=3.0, edge_color='black',
                               arrows=True, arrowsize=20)
        
        # If a negative cycle is provided, highlight it (red dashed)
        if highlight_negative_cycle:
            cycle_edges = [(highlight_negative_cycle[i], highlight_negative_cycle[i+1]) 
                        for i in range(len(highlight_negative_cycle)-1)]
            nx.draw_networkx_edges(self.residual_graph, pos, edgelist=cycle_edges,
                               width=2.5, edge_color='red', style='dashed',
                               arrows=True, arrowsize=20)
        
        # Draw labels
        nx.draw_networkx_labels(self.residual_graph, pos, font_size=12, font_weight='bold')
        nx.draw_networkx_edge_labels(self.residual_graph, pos, edge_labels=edge_labels, 
                                 font_size=10)
        
        # Add title
        if title:
            plt.title(title, fontsize=16)
        else:
            plt.title("Residual Graph for Multi-Object Tracking", fontsize=16)
            
        plt.axis('off')
        plt.tight_layout()
        
        # Save figure
        filename = f"residual_graph_{self.figure_counter:02d}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Saved graph visualization to {filename}")
        plt.close()
    
    def get_trajectories(self):
        """Extract trajectories from the current flow"""
        # Each trajectory is a sequence of fragments
        trajectories = []
        
        # Start with all entry nodes
        entry_nodes = [node for node in self.residual_graph.nodes() if node.startswith('u')]
        visited = set()
        
        for start_node in entry_nodes:
            if start_node in visited:
                continue
            
            # Start a new trajectory
            trajectory = []
            current_node = start_node
            
            # Follow the flow from entry to exit to next entry, etc.
            while current_node not in visited:
                visited.add(current_node)
                
                if current_node.startswith('u'):
                    fragment_id = int(current_node[1:])
                    trajectory.append(fragment_id)
                    
                    # Move to the exit node
                    exit_node = f'v{fragment_id}'
                    
                    # Check if this fragment is included in the flow
                    if (self.residual_graph.has_edge(current_node, exit_node) and 
                        self.residual_graph[current_node][exit_node]['flow'] > 0):
                        current_node = exit_node
                    else:
                        # This fragment is not included
                        break
                else:
                    # We're at an exit node, find the next fragment
                    found_next = False
                    for _, next_node, data in self.residual_graph.out_edges(current_node, data=True):
                        if next_node != 's' and next_node.startswith('u') and data['flow'] > 0:
                            current_node = next_node
                            found_next = True
                            break
                    
                    if not found_next:
                        # End of trajectory
                        break
            
            if len(trajectory) > 0:
                # Check if at least one fragment is included in the flow
                included = False
                for i in range(len(trajectory)):
                    u_node = f'u{trajectory[i]}'
                    v_node = f'v{trajectory[i]}'
                    if (self.residual_graph.has_edge(u_node, v_node) and 
                        self.residual_graph[u_node][v_node]['flow'] > 0):
                        included = True
                        break
                
                if included:
                    trajectories.append(trajectory)
        
        return trajectories

# Example usage
def main():
    tracker = NCCTracker()
    
    # Create some example fragments
    fragments = [
        {'id': 0, 'start_time': 0, 'end_time': 10, 'lane': 1, 'position': [0, 100]},
        {'id': 1, 'start_time': 0, 'end_time': 10, 'lane': 2, 'position': [0, 200]},
        {'id': 2, 'start_time': 15, 'end_time': 25, 'lane': 1, 'position': [200, 300]},
        {'id': 3, 'start_time': 15, 'end_time': 25, 'lane': 2, 'position': [200, 400]}
    ]
    
    # Add fragments one by one
    for fragment in fragments:
        tracker.add_fragment(fragment)
    
    # Visualize the final residual graph
    tracker._visualize_residual_graph(
        title="Final Residual Graph",
        show_min_cost_circulation=True
    )
    
    # Get the final trajectories
    trajectories = tracker.get_trajectories()
    print("\nFinal Trajectories:")
    for i, traj in enumerate(trajectories):
        print(f"Trajectory {i+1}: Fragments {traj}")

if __name__ == "__main__":
    main()


=== Adding Fragment 0 ===
Fragment details: Lane 1, Time 0-10
No negative cycles found.
Saved graph visualization to residual_graph_01.png

=== Adding Fragment 1 ===
Fragment details: Lane 2, Time 0-10
Saved graph visualization to residual_graph_02.png
No negative cycles found.
Saved graph visualization to residual_graph_03.png

=== Adding Fragment 2 ===
Fragment details: Lane 1, Time 15-25
Saved graph visualization to residual_graph_04.png
Found negative cycle: s -> u0 -> v0 -> u2 -> v2 -> s with cost -0.5
Saved graph visualization to residual_graph_05.png
Pushing flow of 1 through cycle: s -> u0 -> v0 -> u2 -> v2 -> s
Saved graph visualization to residual_graph_06.png

=== Adding Fragment 3 ===
Fragment details: Lane 2, Time 15-25
Saved graph visualization to residual_graph_07.png
Found negative cycle: s -> u1 -> v1 -> u3 -> v3 -> s with cost -0.5
Saved graph visualization to residual_graph_08.png
Pushing flow of 1 through cycle: s -> u1 -> v1 -> u3 -> v3 -> s
Saved graph visualizat